**Summary**  

This tutorial demonstrates how to access AppEEARS Cloud Optimized GeoTIFF (COG) outputs. NASA's Application for Extracting and Exploring Analysis Ready Samples ([AρρEEARS](https://appeears.earthdatacloud.nasa.gov/)) has been migrated to NASA's Earthdata Cloud space located in **AWS us-west 2**. This enables the user working in the cloud instance deployed in **AWS us-west 2** to access outputs direcly in the cloud using S3 link returned in the location header of the response. In this tutorial, we will walk through the process of submitting an area sample and accessing a Cloud Optimized GeoTIFF (COG) outputs from AppEEARS. 
The Dixie Fire, the second-largest fire in California history, used as an example in this tutorial. According to [CalFire](https://www.fire.ca.gov/incidents/2021/7/13/dixie-fire/), the fire has started on July 13, 2021 and burned more than 963,276 acres acres. The fire was hundred percent contained by October 2021. On August 18, the Dixie Fire merged with the Morgan Fire, which had been started by lightning August 12, close to Lassen National Park.   

**Requirements**    
- Earthdata Login Authentication is required to access AppEEARS API and AppEEARS outpurs direcrly from an Amazon AWS bucket. See **Requirements** section in **README.md**.

**Learning Objectives**  
- Learn how to access AppEEARS Cloud Optimized GeoTIFF (COG) outputs


**Tutorial Outline**  
   1. Setting Up  
   2. Submit an area request in AppEEARS  
   3. Extract the Direct S3 links  
   4. Create a boto3 Refreshable Session  
   5. Single COG File In-Region Direct S3 Access   
   6. Multiple COG File In-Region Direct S3 Access  
   7. Explore the EVI Time Series   
    
    



## 1. setting up

Import required packages.

In [ ]:
import requests
import getpass, pprint, time, os, cgi, json
import geopandas 
import numpy
import datetime

import os
import io
import json
from urllib import parse
import requests
from netrc import netrc
from uuid import uuid4
import time
from pathlib import Path
from datetime import datetime, timezone
from botocore.client import Config
import rioxarray
import xarray
import hvplot.xarray
import holoviews
import geoviews
import rasterio 
from rasterio.plot import show
from rasterio.session import AWSSession
import s3fs
import pandas
import warnings
import sys
sys.path.append('modules/')
import aws_session
warnings.filterwarnings('ignore')

In order to successfully run this tutorial, it is required to create a .netrc file in your home directory. The function `_validate_netrc` defined in `aws_session` checks if the netrc file with proper format exists in your home directory. If the netrc file does not exist, it will prompt to ask your Earthdata Login username and password and will create a netrc file. Please see the **Prerequisites** section in `README.md`. 

In [ ]:
# validate if netrc file is present else create one via the user / password prompt for the urs.earthdata.nasa.gov
aws_session._validate_netrc()

## 2. Submit an area request in AppEEARS 
In this step, we are going to submit an area request with GeoTIFF as an output format. You can also submit this request using [AppEEARS Graphic User Interface(GUI)](https://appeears.earthdatacloud.nasa.gov/task/area) and upload the JSON file provided in the repository (AppEEARS-Data-Resources/additional_file/Dixie-Fire-request.json). If you have your completed request, save your `task_id` to a variable, skip this step, and move to the next step of tutorial.  

Assign the AρρEEARS API endpoint to a variable. 

In [ ]:
appeears_API_endpoint = 'https://appeears.earthdatacloud.nasa.gov/api/'

To access AppEEARS API a **token** is needed. This token is created using AppEEARS API endpoint, Earthdata Login credential stored in .netrc file, and `requests.post` function. This generated token will be added to the header. The header will be passed to `requests.post` function showing you are a qualifid user for submit and access a request. 

In [ ]:
urs = 'urs.earthdata.nasa.gov'
token_response = requests.post('{}login'.format(appeears_API_endpoint), auth = (netrc().authenticators(urs)[0],netrc().authenticators(urs)[2])).json() # Insert API URL, call login service, provide credentials & return json 
token = token_response['token']                      # Save login token to a variable
head = {'Authorization': 'Bearer {}'.format(token)}  # Create a header to store token information, needed to submit a request

Next, compile a JSON object with the request parameters. Dixie fire, started on July 13, 2021, but we extended the search query to two years to see the time series. The GeoJSON of Region of Interest(ROI) including Lassen National Park region, CA can be downloaded from the repository. For this tutorial, we are requesting `_500m_16_days_EVI` layer of `MOD13A1.061` to see how Enhanced Vegetation Indices (EVI) varies before and after the fire event. Learn more about the MODIS Vegetation Indices 16-Day Version 6.1 product [here](https://doi.org/10.5067/MODIS/MOD13A1.061). Below the AppEEARS search parameters are defined and the `task` JSON  .


In [ ]:
task_name = "Dixie Fire"
task_type = 'area'                  # Type of task, area or point
proj = 'geographic'                 # Set output projection 
outFormat = 'geotiff'               # Set output file format type
startDate = '01-01-2021'            # Start of the date range for which to extract data: MM-DD-YYYY
endDate = '12-31-2022'              # End of the date range for which to extract data: MM-DD-YYYY
ROI =  geopandas.read_file('additional_file/DixieFire.geojson').to_json()
prodLayer = [{'layer': '_500m_16_days_EVI', 'product': 'MOD13A1.061'}]

In [ ]:
task = {
    'task_type': task_type,
    'task_name': task_name,
    'params': {
         'dates': [
         {
             'startDate': startDate,
             'endDate': endDate
         }],
         'layers': prodLayer,
         'output': {
                 'format': {
                         'type': outFormat}, 
                         'projection': proj},
         'geo': json.loads(ROI),
    }
}

Next, submit the AppEEARS request using `post` function from `requests` library.

In [ ]:
task_response = requests.post('{}task'.format(appeears_API_endpoint), json=task, headers=head).json()   # Post json to the API task service, return response as json
task_response                                                                  # Print task response

Save the `task_id` and wait until your request is processed and complete. 

In [ ]:
task_id = task_response['task_id']
task_id

In [ ]:
# Ping API until request is complete, then continue to Section 3
while requests.get('{}task/{}'.format(appeears_API_endpoint, task_id), headers=head).json()['status'] != 'done':
    print(requests.get('{}task/{}'.format(appeears_API_endpoint, task_id), headers=head).json()['status'])
    time.sleep(60)
print(requests.get('{}task/{}'.format(appeears_API_endpoint, task_id), headers=head).json()['status'])

## 3. Extract the Direct S3 links

Now that we have our outputs ready, we can get the bundle information for the files included in the outputs. If you submitted your request using AppEEARS GUI, assign your sample's `task_id` to the variable `task_id` below. 

In [ ]:
task_id = 'a1714bef-bb86-4bd3-be54-bb84c01a11d8'

`requests.get` is used toget the bundle information. Below, bundle information for the first output file is printed. The bundle information includes `s3_url` in addition to the other information such as `file_name`, `file_id`, and `file_type`.  
Each output file can be downloaded using the `file_id` and AppEEARS API endpoint. AppEEARS outputs are stored in an AWS bucket that can be accessed using `S3_url`. 

In [ ]:
bundle = requests.get('{}bundle/{}'.format(appeears_API_endpoint,task_id), headers=head).json()  # Call API and return bundle contents for the task_id as json

bundle['files'][0]

Below, the S3 Links to Cloud Optimized GeoTIFF outputs and then S3 links for EVI layers are filted. 

In [ ]:
cog_urls = [f['s3_url'] for f in bundle['files'] if f['file_type'] == 'tif']
# cog_urls

In [ ]:
EVI_cog_urls = [link for link in cog_urls if '500m_16_days_EVI' in link]
EVI_cog_urls

In the same way, get the links to quality layers. 

In [ ]:
qual_cog_urls = [link for link in cog_urls if '500m_16_days_VI_Quality' in link]


## 4. Create a boto3 Refreshable Session

AppEEARS outputs are freely accessible from a cloud instance in `us-west-2` region. In order to access our output files, a **Boto3 session** is needed. The Boto session will stores the required configurations for an easy integration between Python and AWS services. Below, `get_boto3_refreshable_session` stored in `aws_session` will access  your Earthdata login credentidals store in .netrc file and generate S3 credential by making a call to AppEEARS S3 credential endpoint, and create a boto3 session. This session will be auto-renewed as needed to prevent timeouts errors related to S3 credentials.

In [ ]:
region_name = 'us-west-2'
s3_creds_endpoint = f"{appeears_API_endpoint}/s3credentials"


In [ ]:
# Boto3 Session required by the rioxarray package 
boto3_session = aws_session.get_boto3_refreshable_session(s3_creds_endpoint, region_name)


Below, GDAL and AWS configuration and any environmner options are passed to
`rasterio.env()`.  When the Python context manager is entered  all the configuration options are set and when the context is exited, configuration options are removed.
      

In [ ]:
rio_env = rasterio.Env(
    AWSSession(boto3_session),
    GDAL_DISABLE_READDIR_ON_OPEN='EMPTY_DIR',
    GDAL_HTTP_COOKIEFILE=os.path.expanduser('~/cookies.txt'),
    GDAL_HTTP_COOKIEJAR=os.path.expanduser('~/cookies.txt')
)
rio_env.__enter__()


## 5. Single COG File In-Region Direct S3 Access 

COG datasets can be read using `open_rasterio` function from `rioxarray` library. The coordinates are generated automatically from the file’s geoinformation. 
Below, the first EVI S3 link is read as a `xarray.DataArray`. The data array is scaled and nodata values are masked by setting `mask_and_scale` to `True`. Chunk sizes can be provided or can be set to 'auto' to make a sensible chunk sized according to each dimension. 

In [ ]:
EVI_obj = rioxarray.open_rasterio(EVI_cog_urls[0], chunks = 'auto'  , mask_and_scale = True).squeeze('band', drop=True)

EVI_obj

If the chunk shape and array are the same, the chunk selected for the dataset is bigger in size. you can set the chunk size manually.  

In [ ]:
EVI_obj.chunk(chunks={"x": 5, "y": 5})


To view the values in the data array, you can use `load` function. 

In [ ]:
EVI_obj.load()


Now the data are loaded, lets quickly visualize the first EVI file using `hvplot.image` function. 

In [ ]:
EVI_obj.hvplot.image(x='x', y='y', rasterize=True, title= EVI_cog_urls[0].split('/')[-1], colorbar=True, cmap='YlGnBu').opts(clim=(0, 0.8))

## 6. Multiple COG File In-Region Direct S3 Access

Next, lets access the EVI time series. Here, the EVI time series is being read and concadenated to a single `xarray.DataArray` using the `concat` function.  

Below the date are saved in a list using S3 links.

In [ ]:
time_list = []
for obs in range(len(EVI_cog_urls)):
    doy = EVI_cog_urls[obs].split('_doy')[1].split('_aid')[0]
    doy_date = datetime.strptime(doy, '%Y%j').strftime('%d-%m-%Y')
    time_list.append(doy_date)
time = xarray.Variable('time', time_list)


In [ ]:
chunks=dict(band=1, x=100, y=100)
EVI_series = xarray.concat([rioxarray.open_rasterio(f, chunks = 'auto', mask_and_scale = True).squeeze('band', drop=True) for f in EVI_cog_urls], dim=time)
EVI_series = EVI_series.rename('EVI')
EVI_series

## 7. Explore the EVI Time Series 

Below, a dynamic plot is created to visually look at the variation in EVI through out these two years. You can manually select the dates from the dropdown next to the map. The latest observation before the fire event is on July 12, 2021. and the next observation is on July 28, 2021 which is the first observation after the fire event. A sudden decrease in EVI after July 12, 2021 is visually noticable and the area that vegetation is being removed grows spatially as you look through next observations. If you look at the observattions of the next year, you still can identify the scar left by fire.

In [ ]:
map = EVI_series.hvplot.image(x='x', y='y', bands='time', rasterize=True, colorbar=True, cmap='YlGnBu').opts(clim=(0, 0.8))
map

You can also create a plot showing the EVI time series for a specific latittude and longitude. 

In [ ]:
EVI_series.sel(x=-121.260, y=40.330,method='nearest').hvplot.line(x='time', y='EVI', rot=90, frame_width= 1000, frame_height= 300, fontscale=1.5)


Now, combine the time series map and the plot. Below,a dynamic map is created that shows the EVI time series as you move your mouse to a different location on the map. You can update the base map by selecting the date from the dropdown menu on the far right side. This is an easy way to explore your data before further processing. 

In [ ]:

# Stream of X and Y positional data
posxy = holoviews.streams.PointerXY(source=map, x=-121.4, y=40.03) 

# Function to build a time series using the mouse hover positional information retrieved from the map 
def point_spectra(x,y):
    return EVI_series.sel(x=x,y=y,method='nearest').hvplot.line(x='time', y='EVI', rot=90, frame_width= 800, frame_height= 300, fontscale=1.5) 

# Define the Dynamic Maps
point_dmap = holoviews.DynamicMap(point_spectra, streams=[posxy])

# Plot the Map and Dynamic Map side by side
map + point_dmap

Finally, exit the context manager to remove the configuration options.

In [ ]:
# Exit our context
rio_env.__exit__()


## Contact Info:  

Email: LPDAAC@usgs.gov  
Voice: +1-866-573-3222  
Organization: Land Processes Distributed Active Archive Center (LP DAAC)¹  
Website: <https://lpdaac.usgs.gov/>  
Date last modified: 02-16-2022  

¹Work performed under USGS contract G15PD00467 for NASA contract NNG14HH33I.  